# नमूना 02: OpenAI SDK एकीकरण

यह नोटबुक OpenAI Python SDK के साथ उन्नत एकीकरण को प्रदर्शित करती है, जो Microsoft Foundry Local और Azure OpenAI दोनों का समर्थन करती है, साथ ही स्ट्रीमिंग प्रतिक्रियाओं और उचित त्रुटि प्रबंधन के साथ।

## अवलोकन

यह नमूना निम्नलिखित को प्रदर्शित करता है:
- Foundry Local और Azure OpenAI के बीच सहज स्विचिंग
- बेहतर उपयोगकर्ता अनुभव के लिए स्ट्रीमिंग चैट पूर्णता
- FoundryLocalManager SDK का उचित उपयोग
- मजबूत त्रुटि प्रबंधन और फॉलबैक तंत्र
- उत्पादन-तैयार कोड पैटर्न


## आवश्यकताएँ

- **Foundry Local**: इंस्टॉल और चालू (स्थानीय इनफेरेंस के लिए)
- **Python**: 3.8 या बाद का संस्करण OpenAI SDK के साथ
- **Azure OpenAI**: मान्य एंडपॉइंट और API कुंजी (क्लाउड इनफेरेंस के लिए)

### डिपेंडेंसीज़ इंस्टॉल करें


In [ ]:
# Install required packages
!pip install openai foundry-local-sdk

## लाइब्रेरी आयात करें और सेटअप करें


In [ ]:
import os
import sys
from openai import OpenAI
import time
from typing import Tuple

try:
    from foundry_local import FoundryLocalManager
    FOUNDRY_SDK_AVAILABLE = True
    print("✅ Foundry Local SDK is available")
except ImportError:
    FOUNDRY_SDK_AVAILABLE = False
    print("⚠️ Foundry Local SDK not available, manual configuration will be used")

## कॉन्फ़िगरेशन विकल्प

सही पर्यावरण वेरिएबल्स सेट करके Azure OpenAI (क्लाउड) या Foundry Local (डिवाइस पर) के बीच चयन करें।


### विकल्प 1: Azure OpenAI कॉन्फ़िगरेशन

अपनी Azure OpenAI क्रेडेंशियल्स को अनकमेंट करें और सेट करें:


In [ ]:
# Azure OpenAI Configuration
# Uncomment and set your actual values

# os.environ["AZURE_OPENAI_ENDPOINT"] = "https://your-resource.openai.azure.com"
# os.environ["AZURE_OPENAI_API_KEY"] = "your-api-key-here"
# os.environ["AZURE_OPENAI_API_VERSION"] = "2024-08-01-preview"
# os.environ["MODEL"] = "your-deployment-name"  # e.g., "gpt-4"

print("Azure OpenAI configuration ready (if credentials are set)")

### विकल्प 2: फाउंड्री लोकल कॉन्फ़िगरेशन

स्थानीय अनुमान के लिए डिफ़ॉल्ट सेटिंग्स:


In [ ]:
# Foundry Local Configuration (default)
FOUNDRY_MODEL = "phi-4-mini"  # Change to your preferred model
FOUNDRY_BASE_URL = "http://localhost:51211"
FOUNDRY_API_KEY = ""  # Usually empty for local

print(f"Foundry Local configuration ready with model: {FOUNDRY_MODEL}")

## क्लाइंट फैक्ट्री फ़ंक्शन्स

ये फ़ंक्शन्स आपकी कॉन्फ़िगरेशन के आधार पर उपयुक्त OpenAI क्लाइंट बनाते हैं:


In [ ]:
def create_azure_client() -> Tuple[OpenAI, str]:
    """Create Azure OpenAI client."""
    azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
    azure_api_key = os.environ.get("AZURE_OPENAI_API_KEY")
    azure_api_version = os.environ.get("AZURE_OPENAI_API_VERSION", "2024-08-01-preview")
    
    if not azure_endpoint or not azure_api_key:
        raise ValueError("Azure OpenAI endpoint and API key are required")
    
    model = os.environ.get("MODEL", "your-deployment-name")
    client = OpenAI(
        base_url=f"{azure_endpoint}/openai",
        api_key=azure_api_key,
        default_query={"api-version": azure_api_version},
    )
    
    print(f"🌐 Azure OpenAI client created with model: {model}")
    return client, model


def create_foundry_client() -> Tuple[OpenAI, str]:
    """Create Foundry Local client with SDK management."""
    alias = FOUNDRY_MODEL
    
    if FOUNDRY_SDK_AVAILABLE:
        try:
            # Use FoundryLocalManager for proper service management
            print(f"🔄 Initializing Foundry Local with model: {alias}...")
            manager = FoundryLocalManager(alias)
            model_info = manager.get_model_info(alias)
            
            # Configure OpenAI client to use local Foundry service
            client = OpenAI(
                base_url=manager.endpoint,
                api_key=manager.api_key  # API key is not required for local usage
            )
            
            print(f"✅ Foundry Local SDK initialized")
            print(f"   Endpoint: {manager.endpoint}")
            print(f"   Model: {model_info.id}")
            return client, model_info.id
        except Exception as e:
            print(f"⚠️ Could not use Foundry SDK ({e}), falling back to manual configuration")
    
    # Fallback to manual configuration
    client = OpenAI(
        base_url=f"{FOUNDRY_BASE_URL}/v1",
        api_key=FOUNDRY_API_KEY
    )
    
    print(f"🔧 Manual Foundry Local configuration")
    print(f"   Endpoint: {FOUNDRY_BASE_URL}/v1")
    print(f"   Model: {alias}")
    return client, alias

## क्लाइंट प्रारंभ करें

यह स्वचालित रूप से पहचानता है कि Azure OpenAI या Foundry Local का उपयोग करना है:


In [ ]:
def initialize_client() -> Tuple[OpenAI, str, str]:
    """Initialize the appropriate OpenAI client."""
    
    # Check for Azure OpenAI configuration
    azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
    azure_api_key = os.environ.get("AZURE_OPENAI_API_KEY")
    
    if azure_endpoint and azure_api_key:
        print("🌐 Azure OpenAI configuration detected")
        try:
            client, model = create_azure_client()
            return client, model, "azure"
        except Exception as e:
            print(f"❌ Azure OpenAI initialization failed: {e}")
            print("🔄 Falling back to Foundry Local...")
    
    # Use Foundry Local
    print("🏠 Using Foundry Local configuration")
    try:
        client, model = create_foundry_client()
        return client, model, "foundry"
    except Exception as e:
        print(f"❌ Foundry Local initialization failed: {e}")
        raise

# Initialize the client
print("Initializing OpenAI client...")
print("=" * 50)
client, model, provider = initialize_client()
print("=" * 50)
print(f"✅ Initialization complete! Using {provider} with model: {model}")

## बेसिक चैट पूर्णता

एक साधारण चैट पूर्णता का परीक्षण करें:


In [ ]:
def simple_chat(prompt: str, max_tokens: int = 150) -> str:
    """Send a simple chat message and get response."""
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {e}"

# Test basic chat
test_prompt = "Say hello from the SDK quickstart and explain what you are in one sentence."

print(f"👤 User: {test_prompt}")
print("\n🤖 Assistant:")
response = simple_chat(test_prompt)
print(response)

## स्ट्रीमिंग चैट पूर्णता

बेहतर उपयोगकर्ता अनुभव के लिए स्ट्रीमिंग प्रतिक्रियाओं का प्रदर्शन करें:


In [ ]:
def streaming_chat(prompt: str, max_tokens: int = 300) -> str:
    """Send a chat message with streaming response."""
    try:
        print("🤖 Assistant (streaming):")
        
        stream = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            stream=True
        )
        
        full_response = ""
        for chunk in stream:
            if chunk.choices[0].delta.content is not None:
                content = chunk.choices[0].delta.content
                print(content, end="", flush=True)
                full_response += content
        
        print("\n")  # New line after streaming
        return full_response
    except Exception as e:
        error_msg = f"Error: {e}"
        print(error_msg)
        return error_msg

# Test streaming chat
streaming_prompt = "Explain the key benefits of using Microsoft Foundry Local for AI development. Include aspects like privacy, performance, and cost."

print(f"👤 User: {streaming_prompt}\n")
streaming_response = streaming_chat(streaming_prompt)

## बहु-चरण वार्तालाप

वार्तालाप के संदर्भ को बनाए रखने का प्रदर्शन करें:


In [ ]:
class ConversationManager:
    """Manages multi-turn conversations with context."""
    
    def __init__(self, system_prompt: str = None):
        self.messages = []
        if system_prompt:
            self.messages.append({"role": "system", "content": system_prompt})
    
    def send_message(self, user_message: str, max_tokens: int = 200) -> str:
        """Send a message and get response while maintaining context."""
        # Add user message to conversation
        self.messages.append({"role": "user", "content": user_message})
        
        try:
            response = client.chat.completions.create(
                model=model,
                messages=self.messages,
                max_tokens=max_tokens
            )
            
            assistant_message = response.choices[0].message.content
            
            # Add assistant response to conversation
            self.messages.append({"role": "assistant", "content": assistant_message})
            
            return assistant_message
        except Exception as e:
            return f"Error: {e}"
    
    def get_conversation_length(self) -> int:
        """Get the number of messages in the conversation."""
        return len(self.messages)

# Create conversation manager with system prompt
system_prompt = "You are a helpful AI assistant specialized in explaining AI and machine learning concepts. Be concise but informative."
conversation = ConversationManager(system_prompt)

# Multi-turn conversation example
conversation_turns = [
    "What is the difference between AI inference on-device vs in the cloud?",
    "Which approach is better for privacy?",
    "What about performance and latency considerations?"
]

for i, turn in enumerate(conversation_turns, 1):
    print(f"\n{'='*60}")
    print(f"Turn {i}")
    print(f"{'='*60}")
    print(f"👤 User: {turn}")
    
    response = conversation.send_message(turn)
    print(f"\n🤖 Assistant: {response}")

print(f"\n📊 Conversation summary: {conversation.get_conversation_length()} messages total")

## प्रदर्शन तुलना

विभिन्न परिदृश्यों के लिए प्रतिक्रिया समय की तुलना करें:


In [ ]:
def benchmark_response_time(prompt: str, iterations: int = 3) -> dict:
    """Benchmark response time for a given prompt."""
    times = []
    responses = []
    
    for i in range(iterations):
        start_time = time.time()
        
        try:
            response = client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": prompt}],
                max_tokens=50  # Keep responses short for timing
            )
            
            end_time = time.time()
            response_time = end_time - start_time
            
            times.append(response_time)
            responses.append(response.choices[0].message.content)
            
        except Exception as e:
            print(f"Error in iteration {i+1}: {e}")
    
    if times:
        avg_time = sum(times) / len(times)
        min_time = min(times)
        max_time = max(times)
        
        return {
            "average_time": avg_time,
            "min_time": min_time,
            "max_time": max_time,
            "all_times": times,
            "sample_response": responses[0] if responses else None
        }
    
    return {"error": "No successful responses"}

# Benchmark different types of prompts
benchmark_prompts = [
    "What is AI?",
    "Explain machine learning in simple terms.",
    "List 3 benefits of edge computing."
]

print(f"⏱️  Performance Benchmark ({provider} - {model})")
print("=" * 60)

for prompt in benchmark_prompts:
    print(f"\n📝 Prompt: '{prompt}'")
    results = benchmark_response_time(prompt)
    
    if "error" not in results:
        print(f"   ⏰ Average time: {results['average_time']:.2f}s")
        print(f"   ⚡ Fastest: {results['min_time']:.2f}s")
        print(f"   🐌 Slowest: {results['max_time']:.2f}s")
        print(f"   📄 Sample response: {results['sample_response'][:100]}...")
    else:
        print(f"   ❌ {results['error']}")

## उन्नत कॉन्फ़िगरेशन और त्रुटि प्रबंधन

विभिन्न पैरामीटर और त्रुटि परिदृश्यों का परीक्षण करें:


In [ ]:
def test_different_parameters():
    """Test chat completions with different parameters."""
    prompt = "Write a creative short story about AI."
    
    # Test different temperature values
    temperatures = [0.1, 0.5, 0.9]
    
    for temp in temperatures:
        print(f"\n🌡️ Temperature: {temp}")
        print("-" * 30)
        
        try:
            response = client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": prompt}],
                max_tokens=100,
                temperature=temp
            )
            
            print(f"Response: {response.choices[0].message.content[:150]}...")
            
        except Exception as e:
            print(f"Error with temperature {temp}: {e}")

test_different_parameters()

## सेवा स्वास्थ्य जांच

सेवा की व्यापक स्वास्थ्य और क्षमता जांच:


In [ ]:
def comprehensive_health_check():
    """Perform comprehensive health check of the service."""
    print("🏥 Comprehensive Health Check")
    print("=" * 50)
    
    # 1. Check model listing
    try:
        models_response = client.models.list()
        available_models = [m.id for m in models_response.data]
        print(f"✅ Model listing: SUCCESS")
        print(f"   📋 Available models: {available_models}")
        
        if model in available_models:
            print(f"   ✅ Current model '{model}' is available")
        else:
            print(f"   ⚠️ Current model '{model}' not found in available models")
    except Exception as e:
        print(f"❌ Model listing: FAILED - {e}")
    
    # 2. Test basic completion
    try:
        test_response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": "Say 'Health check successful'"}],
            max_tokens=10
        )
        print(f"✅ Basic completion: SUCCESS")
        print(f"   💬 Response: {test_response.choices[0].message.content}")
    except Exception as e:
        print(f"❌ Basic completion: FAILED - {e}")
    
    # 3. Test streaming
    try:
        stream = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": "Count to 3"}],
            max_tokens=20,
            stream=True
        )
        
        stream_content = ""
        chunk_count = 0
        for chunk in stream:
            if chunk.choices[0].delta.content:
                stream_content += chunk.choices[0].delta.content
                chunk_count += 1
        
        print(f"✅ Streaming: SUCCESS")
        print(f"   📦 Chunks received: {chunk_count}")
        print(f"   💬 Streamed content: {stream_content.strip()}")
    except Exception as e:
        print(f"❌ Streaming: FAILED - {e}")
    
    # 4. Provider-specific information
    print(f"\n📊 Configuration Summary:")
    print(f"   🏢 Provider: {provider}")
    print(f"   🤖 Model: {model}")
    if provider == "foundry":
        print(f"   🏠 Foundry SDK Available: {FOUNDRY_SDK_AVAILABLE}")
        print(f"   🔗 Base URL: {FOUNDRY_BASE_URL}")
    elif provider == "azure":
        print(f"   🌐 Azure Endpoint: {os.environ.get('AZURE_OPENAI_ENDPOINT', 'Not set')}")
        print(f"   🔑 API Version: {os.environ.get('AZURE_OPENAI_API_VERSION', 'Not set')}")

comprehensive_health_check()

## इंटरएक्टिव परीक्षण

इस सेल का उपयोग अपनी खुद की प्रॉम्प्ट्स को इंटरएक्टिव तरीके से परीक्षण करने के लिए करें:


In [ ]:
# Interactive testing - modify the prompt below
custom_prompt = "Explain the concept of 'edge AI' and why it's becoming important."
use_streaming = True  # Set to False for regular completion

print(f"👤 Custom Prompt: {custom_prompt}\n")

if use_streaming:
    custom_response = streaming_chat(custom_prompt, max_tokens=250)
else:
    custom_response = simple_chat(custom_prompt, max_tokens=250)
    print(f"🤖 Assistant: {custom_response}")

## सारांश और अगले कदम

इस नोटबुक ने OpenAI SDK के उन्नत इंटीग्रेशन को प्रदर्शित किया:

### ✅ शामिल प्रमुख विशेषताएं

1. **मल्टी-प्रोवाइडर सपोर्ट**: Azure OpenAI और Foundry Local के बीच सहज स्विचिंग
2. **स्ट्रीमिंग प्रतिक्रियाएं**: बेहतर उपयोगकर्ता अनुभव के लिए रियल-टाइम टोकन जनरेशन
3. **संवाद प्रबंधन**: संदर्भ के साथ मल्टी-टर्न बातचीत
4. **प्रदर्शन बेंचमार्किंग**: प्रतिक्रिया समय मापन और विश्लेषण
5. **व्यापक स्वास्थ्य जांच**: सेवा सत्यापन और डायग्नोस्टिक्स
6. **त्रुटि प्रबंधन**: मजबूत त्रुटि प्रबंधन और फॉलबैक तंत्र

### 🏆 Foundry Local बनाम Azure OpenAI

| पहलू | Foundry Local | Azure OpenAI |
|--------|---------------|---------------|
| **गोपनीयता** | ✅ डेटा स्थानीय रहता है | ⚠️ डेटा क्लाउड में भेजा जाता है |
| **लेटेंसी** | ✅ कम (स्थानीय इंफेरेंस) | ⚠️ अधिक (नेटवर्क निर्भरता) |
| **लागत** | ✅ मुफ्त (हार्डवेयर के बाद) | 💰 प्रति टोकन भुगतान |
| **ऑफलाइन** | ✅ ऑफलाइन काम करता है | ❌ इंटरनेट की आवश्यकता |
| **मॉडल विविधता** | ⚠️ सीमित चयन | ✅ पूर्ण मॉडल एक्सेस |
| **स्केलिंग** | ⚠️ हार्डवेयर पर निर्भर | ✅ असीमित स्केलिंग |

### 🚀 अगले कदम

- **सैंपल 04**: Chainlit चैट एप्लिकेशन बनाना
- **सैंपल 05**: मल्टी-एजेंट ऑर्केस्ट्रेशन सिस्टम
- **सैंपल 06**: इंटेलिजेंट मॉडल रूटिंग
- **प्रोडक्शन डिप्लॉयमेंट**: स्केलिंग और मॉनिटरिंग पर विचार

### 💡 सर्वोत्तम प्रथाएं

1. **प्रोवाइडर्स के बीच हमेशा फॉलबैक तंत्र लागू करें**
2. **लंबी प्रतिक्रियाओं के लिए स्ट्रीमिंग का उपयोग करें** ताकि प्रदर्शन बेहतर लगे
3. **प्रोडक्शन एप्लिकेशन के लिए उचित त्रुटि प्रबंधन लागू करें**
4. **विभिन्न प्रोवाइडर्स के लिए प्रतिक्रिया समय और लागत की निगरानी करें**
5. **अपनी विशिष्ट आवश्यकताओं के आधार पर सही प्रोवाइडर चुनें**
